In [161]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [162]:
training_dataset = pd.read_csv("train.csv")

In [163]:
training_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [164]:
training_dataset.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [165]:
training_dataset.drop(columns=["PassengerId", "Name", "Cabin", "Ticket"], inplace=True)

In [166]:
training_dataset["Embarked"] = training_dataset["Embarked"].fillna("Other")

In [167]:
training_dataset["Age"] = training_dataset["Age"].fillna(training_dataset["Age"].mean())

In [168]:
training_dataset = pd.concat([training_dataset, 
                              pd.get_dummies(training_dataset["Embarked"]), 
                              pd.get_dummies(training_dataset["Sex"]),
                             ], axis=1)

In [169]:
training_dataset.drop(columns=["Embarked", "Sex"], inplace=True)

In [170]:
Y = training_dataset.loc[:, "Survived"]
X = training_dataset.iloc[:, 1:]

In [171]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True)

In [172]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [173]:
xgb_cls = XGBClassifier()

In [174]:
xgb_cls.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [175]:
from sklearn.metrics import accuracy_score

In [176]:
y_pred = xgb_cls.predict(X_test)

In [177]:
accuracy_score(y_test, y_pred)

0.8071748878923767

In [139]:
test_dataset = pd.read_csv("test.csv")

In [127]:
test_df = test_dataset.drop(columns=["PassengerId", "Name", "Age", "Cabin", "Ticket"])

In [128]:
test_df["Embarked"] = test_df["Embarked"].fillna("Other")

In [129]:
test_df = pd.concat([test_df, 
                              pd.get_dummies(test_df["Embarked"]), 
                              pd.get_dummies(test_df["Sex"]),
                             ], axis=1)

In [130]:
test_df.drop(columns=["Embarked", "Sex"], inplace=True)

In [131]:
test_df["Other"] = 0

In [132]:
test_df.head()

,Pclass,SibSp,Parch,Fare,C,Q,S,female,male,Other
0,3,0,0,7.8292,0,1,0,0,1,0
1,3,1,0,7.0000,0,0,1,1,0,0
2,2,0,0,9.6875,0,1,0,0,1,0
3,3,0,0,8.6625,0,0,1,0,1,0
4,3,1,1,12.2875,0,0,1,1,0,0


In [133]:
original_dataset = pd.read_csv("train.csv")
original_dataset.drop(columns=["PassengerId", "Name", "Age", "Cabin", "Ticket"], inplace=True)
original_dataset["Embarked"] = original_dataset["Embarked"].fillna("Other")
original_dataset = pd.concat([original_dataset, 
                              pd.get_dummies(original_dataset["Embarked"]), 
                              pd.get_dummies(original_dataset["Sex"]),
                             ], axis=1)
original_dataset.drop(columns=["Embarked", "Sex"], inplace=True)

In [135]:
X_dev = test_df.iloc[:, :]

In [136]:
X_dev = scaler.transform(X_dev)

In [137]:
test_df["Survived"] = xgb_cls.predict(X_dev)

In [138]:
test_df

,Pclass,SibSp,Parch,Fare,C,Q,S,female,male,Other,Survived
0,3,0,0,7.8292,0,1,0,0,1,0,1
1,3,1,0,7.0000,0,0,1,1,0,0,0
2,2,0,0,9.6875,0,1,0,0,1,0,1
3,3,0,0,8.6625,0,0,1,0,1,0,1
4,3,1,1,12.2875,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
413,3,0,0,8.0500,0,0,1,0,1,0,1
414,1,0,0,108.9000,1,0,0,1,0,0,1
415,3,0,0,7.2500,0,0,1,0,1,0,1
416,3,0,0,8.0500,0,0,1,0,1,0,1


In [142]:
test_dataset.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [143]:
test_df["PassengerId"] = test_dataset["PassengerId"]

In [146]:
test_df[["PassengerId", "Survived"]].to_csv("submission.csv", index=False)

In [147]:
len(original_dataset)

891